In [1]:
import os
import json
import random

In [2]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\first_epoch_trainset_v7.json', 'r') as f1:
    first = json.loads(f1.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\second_epoch_trainset_v3.json', 'r') as f2:
    second = json.loads(f2.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\third_epoch_trainset_v3.json', 'r') as f3:
    third = json.loads(f3.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\forth_epoch_trainset_v3.json', 'r') as f4:
    forth = json.loads(f4.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\fifth_epoch_trainset_v3.json', 'r') as f5:
    fifth = json.loads(f5.read())

final_trainset = first + second + third + forth + fifth
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\ner_trainset_v1.json', 'w') as f:
    f.write(json.dumps(final_trainset))

In [3]:
len(final_trainset)

4147

In [2]:
def remove_unlabled_shots_from_corpus(corpus: dict, blank_seq: list):
    for n in blank_seq:
        for idx in range(n * 8, (n + 1) * 8):
            try:
                del corpus[str(idx)]
            except:
                pass
    return corpus

def remove_null_dict_from_labels(corpus_labels: dict, blank_seq: list):
    for n in blank_seq:
        del corpus_labels[n]
    return corpus_labels

def initial_dataset(corpus: dict, corpus_labels: dict):
    dataset_dict = {"corpus": [], "labels": []}
    for key, value in corpus.items():
        dataset_dict['corpus'].append(value)
    for key, value in corpus_labels.items():
        for i in value:
            dataset_dict['labels'].append(i)

    return dataset_dict

In [3]:
def process_labels(dataset_dict: dict):
    dataset_dict['new_labels'] = []
    for i, sentence in enumerate(dataset_dict['corpus']):
        tmp = ['O'] * len(sentence)
        labels = dataset_dict['labels'][i]['output']
        for label in labels:
            for idx in range(label['start'], label['end']):
                tmp[idx] = f'I-{label['type']}'
        dataset_dict['new_labels'].append(tmp)

    return dataset_dict

In [ ]:
with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\corpus\texts_sentence.json", "r", encoding="utf-8") as f:
    corpus = json.loads(f.read())

with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\corpus\corpus_labels.txt", "r", encoding="utf-8") as f:
    corpus_labels = eval(f.read())

In [ ]:
blank_seq = []
for k, v in corpus_labels.items():
    if not v:
        blank_seq.append(k)

In [ ]:
len(corpus)

In [ ]:
new_corpus = remove_unlabled_shots_from_corpus(corpus, blank_seq)
new_corpus_labels = remove_null_dict_from_labels(corpus_labels, blank_seq)

In [ ]:
dataset_dict = initial_dataset(new_corpus, new_corpus_labels)

In [ ]:
dataset_dict['corpus'][0]

In [ ]:
dataset_dict['labels'][0]['output']

In [ ]:
len(dataset_dict['corpus']) == len(dataset_dict['labels'])

- 训练集：验证集：测试集 = 8：1：1
- 先随机从数据集中取出1000条数据
- 对这1000条数据进行校对（设立一个新的标签PCC。没有识别出来的要标记出来，改正识别错的）
- 校对完之后，建立一个词典
- 用这1000条数据进行预训练
- 再拿出1000条数据用第一次训练后的模型进行标注，再进行一次人工标注后，更新词典。到此为止完成第一次迭代，下面重复这个过程，直到训练集的数据全部标注完成

In [ ]:
trainset_slice = int(len(dataset_dict['corpus']) * 0.8)
test_set_sclice = int(len(dataset_dict['corpus']) * 0.1)
validset_slice = len(dataset_dict['corpus']) - trainset_slice - test_set_sclice

In [ ]:
trainset_slice

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset = json.loads(f.read())
all_idx = list(range(trainset_slice))
first_epoch_idx = first_epoch_trainset['ids']
second_epoch_idx = list(
    random.sample([i for i in all_idx if i not in first_epoch_idx], 1000)
)
third_epoch_idx = list(
    random.sample([i for i in all_idx if i not in (first_epoch_idx + second_epoch_idx)], 1000)
)
forth_epoch_idx = list(
    random.sample([i for i in all_idx if i not in (first_epoch_idx + second_epoch_idx + third_epoch_idx)], 1000)
)
fifth_epoch_idx = list(set(all_idx) - set(first_epoch_idx) - set(second_epoch_idx) - set(third_epoch_idx) - set(forth_epoch_idx))

In [ ]:
def generate_epoch_trainset(idx, dataset_dict, epoch='first'):
    epoch_trainset = {
        'ids': idx,
        "corpus": [dataset_dict["corpus"][i] for i in idx],
        "labels": [dataset_dict["labels"][i] for i in idx],
    }
    with open(f'G:/python_codes/Principles_of_Computer_Construction_KG/output/{epoch}_epoch_trainset.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(epoch_trainset))

In [ ]:
generate_epoch_trainset(second_epoch_idx, dataset_dict, epoch='second')
generate_epoch_trainset(third_epoch_idx, dataset_dict, epoch='third')
generate_epoch_trainset(forth_epoch_idx, dataset_dict, epoch='forth')
generate_epoch_trainset(fifth_epoch_idx, dataset_dict, epoch='fifth')

In [ ]:
(len(fifth_epoch_idx) + len(forth_epoch_idx) + len(third_epoch_idx) + len(second_epoch_idx) + len(first_epoch_idx)) == len(all_idx)

In [ ]:
test_set = {
        'corpus': dataset_dict['corpus'][trainset_slice:trainset_slice + test_set_sclice],
        'labels': dataset_dict['labels'][trainset_slice:trainset_slice + test_set_sclice],
    }
validset = {
        'corpus': dataset_dict['corpus'][trainset_slice + test_set_sclice:],
        'labels': dataset_dict['labels'][trainset_slice + test_set_sclice:],
    }

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(test_set))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(validset))

In [ ]:
first_epoch_trainset_idx = list(
    random.sample(range(0, trainset_slice), 1000)
)
first_epoch_trainset = {
    'ids': first_epoch_trainset_idx,
    "corpus": [dataset_dict["corpus"][i] for i in first_epoch_trainset_idx],
    "labels": [dataset_dict["labels"][i] for i in first_epoch_trainset_idx],
}
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(first_epoch_trainset))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v2.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset = json.loads(f.read())

In [2]:
json_template = {
    "data": {"text": None},
    "predictions": [
        {
            "model_version": "nlp_raner_named-entity-recognition_chinese-large-generic",
            "score": 0.94,
            "result": [],
        }
    ],
}
result_template = {
        'id': None,
        'from_name': 'label',
        'to_name': 'text',
        "type": "labels",
        "value": {
              "start": None,
              "end": None,
              "score": None,  # prob
              "text": None,  # span
              "labels": [
                None  # type
              ]
            }
    }

In [2]:
def generate_pre_labeled_dataset(dataset: dict):
    pre_labeled_data = []
    id_count = 1

    for i, labels in enumerate(dataset["labels"]):

        json_template_tmp = {
            "data": {"text": None},
            "predictions": [
                {
                    "model_version": "nlp_raner_named-entity-recognition_chinese-large-generic",
                    "score": 0.86,
                    "result": [],
                }
            ],
        }

        json_template_tmp["data"]["text"] = dataset["corpus"][i]

        for label in labels["output"]:

            json_template_tmp["predictions"][0]["result"].append(
                {
                    "id": str(id_count),
                    "from_name": "label",
                    "to_name": "text",
                    "type": "labels",
                    "value": {
                        "start": label["start"],
                        "end": label["end"],
                        "score": label["prob"],  # prob
                        "text": label["span"],  # span
                        "labels": [label["type"]],  # type
                    },
                }
            )
            id_count += 1

        pre_labeled_data.append(json_template_tmp)

    return pre_labeled_data

In [7]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\ner_trainset_v1.json', 'r') as f:
    ner_dataset = json.loads(f.read())

dataset[1]

{'text': '随机替换随机替换策略实际上是不要什么算法，从特定的行位置中随机地选取一行换出即可',
 'id': 3594,
 'label': [{'start': 0, 'end': 4, 'text': '随机替换', 'labels': ['TECH']},
  {'start': 18, 'end': 20, 'text': '算法', 'labels': ['ALG']},
  {'start': 4, 'end': 8, 'text': '随机替换', 'labels': ['TECH']},
  {'start': 26, 'end': 27, 'text': '位', 'labels': ['DATA']}],
 'annotator': 1,
 'annotation_id': 5027,
 'created_at': '2024-08-01T12:11:03.657045Z',
 'updated_at': '2024-08-01T12:11:03.657045Z',
 'lead_time': 2.057}

In [3]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset\validset_v7.json', 'r', encoding='utf-8') as f:
    ner_validset = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\testset\test_set_v7.json', 'r', encoding='utf-8') as f:
    ner_testset = json.loads(f.read())

In [6]:
def generate_re_pre_annotated_dataset(dataset):
    pre_labeled_data = []
    id_count = 1

    for i, item in enumerate(dataset):

        json_template_tmp = {
            "data": {"text": None},
            "predictions": [
                {
                    "model_version": "",
                    "score": 0,
                    "result": [],
                }
            ],
        }

        json_template_tmp["data"]["text"] = dataset[i]['text']

        if item.get('label'):
            for label in item["label"]:
                json_template_tmp["predictions"][0]["result"].append(
                    {
                        "id": str(id_count),
                        "from_name": "label",
                        "to_name": "text",
                        "type": "labels",
                        "value": {
                            "start": label["start"],
                            "end": label["end"],
                            "score": 0.1,  # prob
                            "text": label["text"],  # span
                            "labels": label["labels"],  # type
                        },
                    }
                )
                id_count += 1

        pre_labeled_data.append(json_template_tmp)

    return pre_labeled_data

In [7]:
re_pre_annotated_validset = generate_re_pre_annotated_dataset(ner_validset)
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset\re_pre_annotated_validset.json', 'w') as f:
    f.write(json.dumps(re_pre_annotated_validset))

In [8]:
re_pre_annotated_testset = generate_re_pre_annotated_dataset(ner_testset)
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\testset\re_pre_annotated_testset.json', 'w') as f:
    f.write(json.dumps(re_pre_annotated_testset))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_v2_pre_labeled_data.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(first_epoch_trainset)))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_test_set.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(test_set)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_validset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(validset)))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v2.json', 'r', encoding='utf-8') as f:
    test_set_v2 = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v2.json', 'r', encoding='utf-8') as f:
    validset_v2 = json.loads(f.read())

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_test_set_v2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(test_set_v2)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_validset_v2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(validset_v2)))

In [6]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v6.json', 'r', encoding='utf-8') as f:
    test_set_v6 = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v6.json', 'r', encoding='utf-8') as f:
    validset_v6 = json.loads(f.read())

In [9]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_test_set_v6.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(test_set_v6)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_validset_v6.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(validset_v6)))

In [10]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v6.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset_v6 = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_v6_pre_labeled_data.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(first_epoch_trainset_v6)))

In [3]:
# with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\second_epoch_trainset_v2.json', 'r', encoding='utf-8') as f:
#     second_epoch_trainset_v2 = json.loads(f.read())
# with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\third_epoch_trainset_v2.json', 'r', encoding='utf-8') as f:
#     third_epoch_trainset_v2 = json.loads(f.read())
# with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\forth_epoch_trainset_v2.json', 'r', encoding='utf-8') as f:
#     forth_epoch_trainset_v2 = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\fifth_epoch_trainset_v2.json', 'r', encoding='utf-8') as f:
    fifth_epoch_trainset_v2 = json.loads(f.read())

# 注意输出路径
# with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\second_epoch_v2_pre_labeled_data.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(generate_pre_labeled_dataset(second_epoch_trainset_v2)))
# with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\third_epoch_v2_pre_labeled_data.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(generate_pre_labeled_dataset(third_epoch_trainset_v2)))
# with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\forth_epoch_v2_pre_labeled_data.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(generate_pre_labeled_dataset(forth_epoch_trainset_v2)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\fifth_epoch_v2_pre_labeled_data.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(fifth_epoch_trainset_v2)))

In [ ]:
entity_set = set()
for i, labels in enumerate(first_epoch_trainset['labels']):
    entitys = [label['span'] for label in labels['output']]
    entitys_type = [label['type'] for label in labels['output']]
    for item in dict(zip(entitys, entitys_type)).items():
        entity_set.add(item)

In [ ]:
entity_list = list(entity_set)
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(entity_list))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.txt', 'w', encoding='utf-8') as f:
    f.write(str(entity_list))